In [ ]:
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.mlemodel import MLEModel

class PCIStateSpace(MLEModel):
    def __init__(self, endog):
        # State vector: [M_t, R_t]
        # k_states=2, k_posdef=2
        super().__init__(endog, k_states=2, k_posdef=2, initialization='known', 
                         initial_state_cov=np.eye(2)*1e7) # Diffuse initialization
        
        # Design matrix Z: connects states to observation
        # y_t = [1, 1] * [M_t, R_t]'
        self['design'] = np.array([[1.0, 1.0]])
        
        # Transition matrix T: 
        # [rho, 0]
        # [0,   1]
        self['transition'] = np.eye(2)
        
        # Selection matrix R (identity, as shocks hit both states)
        self['selection'] = np.eye(2)

    def update_params(self, rho, sigma_M, sigma_R):
        # Update transition matrix with rho
        self['transition', 0, 0] = rho
        
        # Update covariance matrix Q
        # [sigma_M^2, 0        ]
        # [0,         sigma_R^2]
        self['state_cov', 0, 0] = sigma_M**2
        self['state_cov', 1, 1] = sigma_R**2

    def loglike_obs(self, rho, sigma_M, sigma_R):
        """Helper to compute Log Likelihood for given parameters."""
        self.update_params(rho, sigma_M, sigma_R)
        # Run the Kalman Filter
        return self.loglike()